In [3]:
import os
from glob import glob
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
## MODIFY THIS
# get path to your folder that holds the txt files
source_files = "C:/Users/jacqu/Downloads/Court Case PDFs/Court Case TXTs"
# outputs a list of all the txt files in the folder
source_file_list = sorted(glob(f"{source_files}/*.txt"))

# creates a list of tuples with an elememt for the source path and
# for the file title
file_data = []
for source_file_path in source_file_list:
    # split might be different, recommend checking with INFO.sample() or .head()
    file_title = source_file_path.split('\\')[-1].split(".txt")[0]
    file_data.append((source_file_path, file_title))

# creating df with the file title as the index and source path as a col
INFO = pd.DataFrame(file_data, columns=['txt_path','file_title'])\
    .set_index('file_title').sort_index()
# attempt at dropping any duplicate files with same file name
# this only works if same file has the SAME NAME
# See Notes below
INFO = INFO[~INFO.index.duplicated(keep='first')]

In [ ]:
# for each document
for doc_idx, txt_path in enumerate(INFO['txt_path']):
    # open up the doc
    my_file = open(INFO.txt_path[0], "r") 
    # read the doc line by line
    narrative_lines = my_file.read()
    # make a df with each row rep. a sentence
    sent_df = pd.DataFrame({'sent_str':nltk.sent_tokenize(narrative_lines)})
    sent_df.sent_str = sent_df.sent_str.str.strip()
    sent_df.index.name = "sent_num"

    # creating the model
    model = CountVectorizer(ngram_range = (2, 2), max_features = 100, stop_words='english')
    # apply the model to the sentence df
    # goal: get bigrams for one document 
    matrix = model.fit_transform(sent_df.sent_str).toarray()
    feature_names = model.get_feature_names_out()
    df_output = pd.DataFrame(data = matrix, columns = feature_names)